In [4]:
import random # This library is helpful for many different situations where randomization is necessary.
import simpy # Modeling and analyzing how events and processes change over time is made possible.
import pandas as pd # To utilize the Pandas library, which offers robust tools for data analysis and manipulation, including pandas as pd in Python. 

durationOfProcess = {
    'loading': 5,
    'machining': 10,
    'assembling': 8,
    'inspecting': 6,
    'packaging': 4
}

periodOfMaintaining = 3
rateOfDamage = 0.1

quantityOfEmployees = {
    'loading': 2,
    'machining': 3,
    'assembling': 4,
    'inspecting': 2,
    'packaging': 3
}

durationOfShift = 8
timeOfSimulation = 100
productQuantity = 2

# It defines the line of manufacture period.
class routeofManufacturing:
    def __init__(self, env):
        self.env = env
        self.periods = {
            period: simpy.Resource(env, capacity=quantityOfEmployees[period]) for period in quantityOfEmployees
        }
        self.repair_crew = simpy.Resource(env, capacity=2)
        self.data = []
        
    # It defines the operations of processing period.
    def operationOfProcess(self, part, product_category):
        periods = ['loading', 'machining', 'assembling', 'inspecting', 'packaging']
        for period in periods:
            duration_process = durationOfProcess[period] * (1 if product_category == 1 else 1.2)
            with self.periods[period].request() as request:
                start = self.env.now
                yield request
                try:
                    yield self.env.timeout(duration_process)
                    if random.random() < rateOfDamage:
                        yield self.env.process(self.machineUpkeep(period))
                    finish = self.env.now
                    self.data.append({
                        'Part': part,
                        'Period': period,
                        'Start': start,
                        'Finish': finish,
                        'Duration': finish - start,
                        'Product Category': product_category
                    })
                except simpy.Interrupt:
                    yield self.env.process(self.machineUpkeep(period))

    def machineUpkeep(self, period):
        with self.repair_crew.request() as repair:
            start_repair = self.env.now
            yield repair
            yield self.env.timeout(periodOfMaintaining)

def manufacturerProcess(env, line, part_id, product_category):
    yield env.process(line.operationOfProcess(part_id, product_category))

def setup(env, num_parts_per_type):
    # Manufacturing line is created.
    line = routeofManufacturing(env)

    # For product categories, this is the transaction part.
    for product_category in range(1, productQuantity + 1):
        # It is the part production process initiation part for each product category.
        for i in range(num_parts_per_type):
            part_id = f"Part_{i + 1}"
            env.process(manufacturerProcess(env, line, part_id, product_category))

    # Waiting for the duration of the simulation.
    yield env.timeout(timeOfSimulation)

    # This is the part of extracting data from the production line and creating a DataFrame.
    df = pd.DataFrame(line.data)
    
    # This is the part of rotating the DataFrame.
    return df
# This is the part of creating the SimPy environment.
env = simpy.Environment()
# This is the part of starting the setup function as a process.
process = env.process(setup(env, 5))
# It runs the simulation
env.run()
# We take the DataFrame obtained as a result of the process and print it.
df = process.value
print(df)

      Part      Period  Start  Finish  Duration  Product Category
0   Part_2     loading    0.0     5.0       5.0                 1
1   Part_1     loading    0.0     8.0       8.0                 1
2   Part_3     loading    0.0    10.0      10.0                 1
3   Part_4     loading    0.0    13.0      13.0                 1
4   Part_2   machining    5.0    15.0      10.0                 1
5   Part_5     loading    0.0    15.0      15.0                 1
6   Part_1   machining    8.0    18.0      10.0                 1
7   Part_1     loading    0.0    19.0      19.0                 2
8   Part_2     loading    0.0    21.0      21.0                 2
9   Part_2  assembling   15.0    23.0       8.0                 1
10  Part_3   machining   10.0    23.0      13.0                 1
11  Part_4   machining   13.0    25.0      12.0                 1
12  Part_3     loading    0.0    25.0      25.0                 2
13  Part_1  assembling   18.0    26.0       8.0                 1
14  Part_4